Neil and Fenix worked on this (along with other aspects of final) together in the computer lab
12/5/2021 - 2:00-6:00pm

In [ ]:
#Importing important libraries
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import sep
import astropy
from astropy.io import fits
from matplotlib import rcParams
rcParams['figure.figsize'] = [10., 8.]

In [ ]:
#Opening the 3 fits files
hdu_list_1 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
hdu_list_2 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
hdu_list_3 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")

#print(hdu_list[0].header)

#Printing out information about the 3 images
#Image 1
hdu_list_1.info()
image_data_1 = hdu_list_1[0].data #Initialized variable with image array
image_data_1 = image_data_1.byteswap().newbyteorder()  #NOT NEEDED IF NOT USING SEP?
print(image_data_1.shape)


#Image 2
hdu_list_2.info()
image_data_2 = hdu_list_2[0].data
image_data_2 = image_data_2.byteswap().newbyteorder()
print(image_data_2.shape)
print(image_data_2)

#Image 3
hdu_list_3.info()
image_data_3 = hdu_list_3[0].data
image_data_3 = image_data_3.byteswap().newbyteorder()
print(image_data_3.shape)

#dont think we need to reshape, all (3600,3600)

In [ ]:
#Showing the original images
m_1, s_1 = np.mean(image_data_1), np.std(image_data_1)
m_2, s_2 = np.mean(image_data_2), np.std(image_data_2)
m_3, s_3 = np.mean(image_data_3), np.std(image_data_3)

f, ax = plt.subplots(1,3,figsize=(21,7))
ax[0].imshow(image_data_1, interpolation='nearest', cmap='gray', vmin=m_1-s_1, vmax=m_1+s_1, origin='lower')  #value
ax[1].imshow(image_data_2, interpolation='nearest', cmap='gray', vmin=m_2-s_2, vmax=m_2+s_2, origin='lower')  #hue
ax[2].imshow(image_data_3, interpolation='nearest', cmap='gray', vmin=m_3-s_3, vmax=m_3+s_3, origin='lower')  #saturation


In [ ]:
#Calculating background
bkg_1 = sep.Background(image_data_1)
bkg_2 = sep.Background(image_data_2)
bkg_3 = sep.Background(image_data_3)

#Background subtracted images
# subtract the background
data_sub_1 = image_data_1 - bkg_1
data_sub_2 = image_data_2 - bkg_2
data_sub_3 = image_data_3 - bkg_3
print(data_sub_2)

#Showing the background subtracted images
m_1, s_1 = np.mean(data_sub_1), np.std(data_sub_1)
m_2, s_2 = np.mean(data_sub_2), np.std(data_sub_2)
m_3, s_3 = np.mean(data_sub_3), np.std(data_sub_3)

f, ax = plt.subplots(1,3,figsize=(21,7))
ax[0].imshow(data_sub_1, interpolation='nearest', cmap='gray', vmin=m_1-s_1, vmax=m_1+s_1, origin='lower')  #value
ax[1].imshow(data_sub_2, interpolation='nearest', cmap='gray', vmin=m_2-s_2, vmax=m_2+s_2, origin='lower')  #hue
ax[2].imshow(data_sub_3, interpolation='nearest', cmap='gray', vmin=m_3-s_3, vmax=m_3+s_3, origin='lower')  #saturation


In [ ]:
#attempt at compressing the range of data to scale from 0 to 1
#not working

#Data Sub 1
print(data_sub_1)
ds1_min = min(data_sub_1)
print(ds1_min)
ds1_max = min(data_sub_2)
print(ds1_max)
NewMin = 0
NewMax = 1
OldRange = (ds1_max - ds1_min)  
NewRange = (NewMax - NewMin)  
NewDS1 = (((data_sub_1 - ds1_min) * NewRange) / OldRange) + NewMin
print(NewDS1)

In [ ]:
hsv_image = np.zeros((3600,3600,3))

hsv_image[:,:,0] = data_sub_1    #image_data_1
hsv_image[:,:,1] = data_sub_2    #image_data_2
hsv_image[:,:,2] = data_sub_3    #image_data_3

rgb_image = colors.hsv_to_rgb(hsv_image)

In [ ]:
f = plt.figure(figsize=(7,7))
plt.imshow(rgb_image,cmap='gray',origin='lower')

# All the cells below are BAD attempts

In [ ]:
#tried to use code from class and adapt to the 3 images
d_min = data_sub_1.min()
d_max = data_sub_1.max()
f_min =data_sub_2.min()
f_max = data_sub_2.max()


v = (data_sub_1 - d_min)/(d_max-d_min)
print(v)
s= 1.0 - (data_sub_2 -f_min)/(f_max-f_min)
print(s)

In [ ]:
print(data_sub_3)
e_min = data_sub_3.min()
print(e_min)
e_max = data_sub_3.max()
print(e_max)

h = 0.8 - 0.2*(data_sub_3 - e_min)/(e_max-e_min) 
print(h)

In [ ]:
f, ax = plt.subplots(1,3,figsize=(14,7))
ax[0].imshow(v)  #value
ax[1].imshow(h)   #hue
ax[2].imshow(s)  #saturation

In [ ]:
#tried to take log to scale values between 0 and 1, does not work for these arrays
#since it involved dividing by zero
I1_log = np.log10(data_sub_1)  
I2_log = np.log10(data_sub_2) 
I3_log = np.log10(data_sub_3) 

In [ ]:
f, ax = plt.subplots(1,3,figsize=(14,7))
ax[0].imshow(I1_log, interpolation='nearest', origin='lower')  #value
ax[1].imshow(I2_log, interpolation='nearest', origin='lower')  #hue
ax[2].imshow(I3_log, interpolation='nearest', origin='lower')  #saturation


In [ ]:
f, ax = plt.subplots(1,3,figsize=(14,7))
ax[0].imshow(I1_log,origin='lower')  #value
ax[1].imshow(I2_log,origin='lower') #hue
ax[2].imshow(I3_log,origin='lower')  #saturation

In [ ]:
#MAY NOT NEED THESE 3


#d_min = I1_log.min()
#d_max = I1_log.max()

#v = (I1_log - d_min)/(d_max-d_min)
#s= 1.0 - (I2_log - d_min)/(d_max-d_min)

In [ ]:
#e_min = I3_log.min()
#print(e_min)
#e_max = I3_log.max()
#print(e_max)

#h = 0.8 - 0.2*(I3_log - e_min)/(e_max-e_min) 

In [ ]:
#f, ax = plt.subplots(1,3,figsize=(14,7))
#ax[0].imshow(image_data_1)  #value
#ax[1].imshow()   #hue
#ax[2].imshow(s)  #saturation

In [ ]:
#plotting the logarithmic
hsv_image_2 = np.zeros((3600,3600,3))

hsv_image[:,:,0] = I1_log
hsv_image[:,:,1] = I2_log
hsv_image[:,:,2] = I3_log

rgb_image_2 = colors.hsv_to_rgb(hsv_image_2)

In [ ]:
f = plt.figure(figsize=(7,7))
plt.imshow(rgb_image_2)